All of the non-`solve_ivp` calculations are set to 50,000 solves. This is for consistency as they either run 10,000 or 100,000 solves.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scipy
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import pandas

import sys
sys.path.append("./modules")
from modules.Plot import setup_rc_params
setup_rc_params()

from modules import Potential, FOM, ROM, FOM_scipy
from modules.ROM import apply_range_factor

In [3]:
np.random.seed(12345)

In [4]:
n = 1000
r = np.linspace(0, 12, n)

l = ll = j = 0
S = 0

number_of_snapshots = 8  # number of snapshots used in Figs. 7 and 8

tolerances = 1e-12  # for solve_ivp

# Minnesota runtimes

In [5]:
# paper's solver
minnesota_potential = Potential.Potential("minnesota-no-const", r, 
                                          l=l, ll=ll, j=j, S=S)
minnesota_solver = FOM.MatrixNumerovSolver(minnesota_potential, use_ab=False)

theta_minnesota = minnesota_potential.default_theta
theta_vec_minnesota = minnesota_solver.theta_args(minnesota_potential.default_theta)

solveivp_minnesota = FOM_scipy.FOM(r, l, ll, j, S, potential_name="minnesota", atol=tolerances, rtol=tolerances)

minnesota_emulator = ROM.Emulator({"V_r": [0, 400], "V_s": [-400, 0]}, 
                                  minnesota_solver,
                                  sampling_method="LHS",
                                  param_pts=200,
                                  snapshot_max=number_of_snapshots, 
                                  use_practically=False,  # get the projections for the G-ROM and the LSPG-ROM
                                  ignore_error_bounds=True,
                                  use_scaled_estimated_error=True,
                                  emulation_method="LSPG-ROM", 
                                  error_estimation_method="LSPG-ROM",
                                  use_einsum_paths=False,
                                  orth_cutoff=1e-12,
                                  cutoff_accuracy=1e-6,
                                  verbose=False)

minnesota_emulator.train()
# minnesota_emulator.convergence_plot(dpi=100)
# minnesota_emulator.emulation_errors_at_theta(dpi=100)

print("\n\n")
print(f"number of snapshots in emulator: {minnesota_emulator.snapshots.shape[1]}")

/Users/joshuamaldonado/Documents/GitHub/greedy-emulators/modules/SpecialFunctions.py:52: RuntimeWarning: invalid value encountered in multiply
  riccati_bessel_G = -rho * spherical_yn(l, rho)
/Users/joshuamaldonado/Documents/GitHub/greedy-emulators/modules/FOM.py:99: RuntimeWarning: invalid value encountered in divide
  self.l_prestore_over_rsq = self.l * (self.l + 1) / (self.r ** 2)
/Users/joshuamaldonado/Documents/GitHub/greedy-emulators/modules/gram_schmidt.py:79: NumbaPerformanceWarning: np.vdot() is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  orthogonal_matrix[:, k] -= np.vdot(orthonormal_matrix[:, j],





number of snapshots in emulator: 8


In [6]:
print("FOM runtimes:")
print("solve_ivp runtime:")
solveivp_timeit_MN = %timeit -o solveivp_minnesota.solve(theta_minnesota)
solveivp_avg_MN = solveivp_timeit_MN.average

print("Numerov runtime:")
numerov_timeit_MN = %timeit -o -n 50000 minnesota_emulator.FOM(theta_minnesota, theta_vec=theta_vec_minnesota)
numerov_avg_MN = numerov_timeit_MN.average

print("\n\nROM runtimes:")
print("Numerov G-ROM runtime:")
GROM_timeit_MN = %timeit -o -n 50000 minnesota_emulator.emulate(theta_minnesota, theta_vec=theta_vec_minnesota, emulation_method="G-ROM", estimate_error=False)
GROM_avg_MN = GROM_timeit_MN.average
print("Numerov G-ROM w/ error runtime:")
GROM_error_timeit_MN = %timeit -o -n 50000 minnesota_emulator.emulate(theta_minnesota, theta_vec=theta_vec_minnesota, emulation_method="G-ROM", estimate_error=True, error_estimation_method="G-ROM")
GROM_error_avg_MN = GROM_error_timeit_MN.average

print("Numerov LSPG-ROM runtime:")
LSPGROM_timeit_MN = %timeit -o -n 50000 minnesota_emulator.emulate(theta_minnesota, theta_vec=theta_vec_minnesota, emulation_method="LSPG-ROM", estimate_error=False)
LSPGROM_avg_MN = LSPGROM_timeit_MN.average
print("Numerov LSPG-ROM w/ error runtime:")
LSPGROM_error_timeit_MN = %timeit -o -n 50000 minnesota_emulator.emulate(theta_minnesota, theta_vec=theta_vec_minnesota, emulation_method="LSPG-ROM", estimate_error=True, error_estimation_method="LSPG-ROM")
LSPGROM_error_avg_MN = LSPGROM_error_timeit_MN.average

FOM runtimes:
solve_ivp runtime:
32.5 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Numerov runtime:
90.5 µs ± 705 ns per loop (mean ± std. dev. of 7 runs, 50,000 loops each)


ROM runtimes:
Numerov G-ROM runtime:
8.88 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 50,000 loops each)
Numerov G-ROM w/ error runtime:
25.4 µs ± 334 ns per loop (mean ± std. dev. of 7 runs, 50,000 loops each)
Numerov LSPG-ROM runtime:
22.8 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 50,000 loops each)
Numerov LSPG-ROM w/ error runtime:
24.5 µs ± 415 ns per loop (mean ± std. dev. of 7 runs, 50,000 loops each)


In [7]:
print("Minnesota: \n")

print(f"      Method      |  Speedup  ")
print(f"-------------------------------")
print(f"     solve_ivp    |  {numerov_avg_MN / solveivp_avg_MN:.2e}")
print(f"      Numerov     |  {numerov_avg_MN / numerov_avg_MN:.3}")
print(f"-------------------------------")
print(f"       G-ROM      |  {numerov_avg_MN / GROM_avg_MN:.3}")
print(f"  G-ROM w/ error  |  {numerov_avg_MN / GROM_error_avg_MN:.3}")
print(f"     LSPG-ROM     |  {numerov_avg_MN / LSPGROM_avg_MN:.3}")
print(f"LSPG-ROM w/ error |  {numerov_avg_MN / LSPGROM_error_avg_MN:.3}")

Minnesota: 

      Method      |  Speedup  
-------------------------------
     solve_ivp    |  2.79e-03
      Numerov     |  1.0
-------------------------------
       G-ROM      |  10.2
  G-ROM w/ error  |  3.56
     LSPG-ROM     |  3.97
LSPG-ROM w/ error |  3.7


# Chiral runtimes

In [8]:
chiral_potential = Potential.Potential("chiral", r,
                                       l=l, ll=ll, j=j, S=S)
chiral_solver = FOM.MatrixNumerovSolver(chiral_potential, use_ab=False)
chiral_what_to_vary = apply_range_factor(chiral_potential.default_theta, 0.5)
chiral_what_to_vary.pop("C0")

theta_chiral = chiral_potential.default_theta
theta_vec_chiral = chiral_solver.theta_args(chiral_potential.default_theta)

solveivp_chiral = FOM_scipy.FOM(r, l, ll, j, S, potential_name="chiral", atol=tolerances, rtol=tolerances)

chiral_emulator = ROM.Emulator(chiral_what_to_vary, 
                               chiral_solver,
                               sampling_method="LHS",
                               param_pts=200,
                               snapshot_max=number_of_snapshots, 
                               use_practically=False,  # get the projections for the G-ROM and the LSPG-ROM
                               ignore_error_bounds=True,
                               use_scaled_estimated_error=True,
                               emulation_method="LSPG-ROM", 
                               error_estimation_method="LSPG-ROM",
                               use_einsum_paths=False,
                               orth_cutoff=1e-12,
                               cutoff_accuracy=1e-6,
                               verbose=False)

chiral_emulator.train()
# chiral_emulator.convergence_plot(dpi=100)
# chiral_emulator.emulation_errors_at_theta(dpi=100)

print("\n\n")
print(f"number of snapshots in emulator: {chiral_emulator.snapshots.shape[1]}")

ImportError: cannot import name 'chiralPot' from 'chiral_construction' (unknown location)

In [ ]:
# # visual check that solve_ivp agrees with Numerov

# numerov_chi_chiral = chiral_emulator.FOM(theta_chiral)[:-2]
# solveivp_chi_chiral = solveivp_chiral.solve(theta_chiral)


# plt.title("Chiral potential solution")
# plt.plot(r, numerov_chi_chiral,
#            linestyle=(0, (2, 2)), alpha=0.9, label="Numerov")
# plt.plot(r, solveivp_chi_chiral,
#            linestyle=(2, (2, 2)), alpha=0.9, label="solve_ivp")
# plt.legend()
# plt.show()

In [ ]:
print("FOM runtimes:")
print("solve_ivp runtime:")
solveivp_timeit_Ch = %timeit -o solveivp_chiral.solve(theta_chiral)
solveivp_avg_Ch = solveivp_timeit_Ch.average

print("Numerov runtime:")
numerov_timeit_Ch = %timeit -o -n 50000 chiral_emulator.FOM(theta_chiral, theta_vec=theta_vec_chiral)
numerov_avg_Ch = numerov_timeit_Ch.average

print("\n\nROM runtimes:")
print("Numerov G-ROM runtime:")
GROM_timeit_Ch = %timeit -o -n 50000 chiral_emulator.emulate(theta_chiral, theta_vec=theta_vec_chiral, emulation_method="G-ROM", estimate_error=False)
GROM_avg_Ch = GROM_timeit_Ch.average
print("Numerov G-ROM w/ error runtime:")
GROM_error_timeit_Ch = %timeit -o -n 50000 chiral_emulator.emulate(theta_chiral, theta_vec=theta_vec_chiral, emulation_method="G-ROM", estimate_error=True, error_estimation_method="G-ROM")
GROM_error_avg_Ch = GROM_error_timeit_Ch.average

print("Numerov LSPG-ROM runtime:")
LSPGROM_timeit_Ch = %timeit -o -n 50000 chiral_emulator.emulate(theta_chiral, theta_vec=theta_vec_chiral, emulation_method="LSPG-ROM", estimate_error=False)
LSPGROM_avg_Ch = LSPGROM_timeit_Ch.average
print("Numerov LSPG-ROM w/ error runtime:")
LSPGROM_error_timeit_Ch = %timeit -o -n 50000 chiral_emulator.emulate(theta_chiral, theta_vec=theta_vec_chiral, emulation_method="LSPG-ROM", estimate_error=True, error_estimation_method="LSPG-ROM")
LSPGROM_error_avg_Ch = LSPGROM_error_timeit_Ch.average

In [ ]:
print("Chiral:\n")

print(f"      Method      |  Speedup  ")
print(f"-------------------------------")
print(f"     solve_ivp    |  {numerov_avg_Ch / solveivp_avg_Ch:.2e}")
print(f"      Numerov     |  {numerov_avg_Ch / numerov_avg_Ch:.3}")
print(f"-------------------------------")
print(f"       G-ROM      |  {numerov_avg_Ch / GROM_avg_Ch:.3}")
print(f"  G-ROM w/ error  |  {numerov_avg_Ch / GROM_error_avg_Ch:.3}")
print(f"     LSPG-ROM     |  {numerov_avg_Ch / LSPGROM_avg_Ch:.3}")
print(f"LSPG-ROM w/ error |  {numerov_avg_Ch / LSPGROM_error_avg_Ch:.3}")